In [3]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm

import reverse_geocoder

In [17]:
conflicts = pd.read_csv('20230108-icews-events.tab', sep='\t')
conflicts.head()

,Event ID,Event Date,Source Name,Source Sectors,Source Country,Event Text,CAMEO Code,Intensity,Target Name,Target Sectors,Target Country,Story ID,Sentence Number,Publisher,City,District,Province,Country,Latitude,Longitude
0,37543936,2022-10-13,Saudi Arabia,NaN,Saudi Arabia,Deny responsibility,16,-5.0,United States,NaN,United States,56635750,4,The Oil Daily,NaN,NaN,Washington D.C.,United States,38.9171,-77.0003
1,37543653,2022-10-13,Saudi Arabia,NaN,Saudi Arabia,Praise or endorse,51,3.4,Russia,NaN,Russian Federation,56635751,2,The Oil Daily,Riyadh,NaN,Mintaqat ar Riyad,Saudi Arabia,24.6877,46.7219
2,37542178,2022-10-14,Israel,NaN,Israel,Provide aid,70,7.0,United States,NaN,United States,56633048,5,The Jerusalem Post,NaN,NaN,NaN,United States,38.8951,-77.0364
3,37542179,2022-10-14,United States,NaN,United States,Provide economic aid,71,7.4,Israel,NaN,Israel,56633048,6,The Jerusalem Post,NaN,NaN,NaN,Israel,31.7790,35.2253
4,37542131,2022-10-14,Ukraine,NaN,Ukraine,"Arrest, detain, or charge with legal action",173,-5.0,Volodymyr Zelensky,"Elite,Executive,Executive Office,Government",Ukraine,56633051,2,The Jerusalem Post,NaN,NaN,NaN,Ukraine,50.4547,30.5238


In [18]:
conflicts = conflicts.loc[(df['Intensity'] <= -9.0)]
conflict_countries = conflicts['Target Country'].dropna().unique()

In [5]:
df = pd.read_csv('models/data/grid.csv', encoding="UTF-8")
df.head()

,lat,long,population,wind_speed,solar,country
0,83.166667,-74.208333,0.000009,4.253750,107.335396,CA
1,83.166667,-74.166667,0.000011,4.253750,107.335396,CA
2,83.166667,-74.125000,0.000011,4.253750,107.335396,CA
3,83.166667,-74.083333,0.000011,4.253750,107.335396,CA
4,83.166667,-74.041667,0.000008,4.266758,107.335396,CA


In [6]:
tqdm.pandas()
test = list(df.progress_apply(lambda x: tuple([x['lat'], x['long']]), axis=1))

  2%|▏         | 141253/7550816 [00:04<04:09, 29700.83it/s]


KeyboardInterrupt: 

In [15]:
coordinates = (-37.81, 144.96), (31.76, 35.21)
reverse_geocoder.search(coordinates)

Loading formatted geocoded file...


[{'lat': '-37.814',
  'lon': '144.96332',
  'name': 'Melbourne',
  'admin1': 'Victoria',
  'admin2': 'Melbourne',
  'cc': 'AU'},
 {'lat': '31.76904',
  'lon': '35.21633',
  'name': 'Jerusalem',
  'admin1': 'Jerusalem',
  'admin2': '',
  'cc': 'IL'}]

In [22]:
def get_country(coordinates):
    return reverse_geocoder.search(coordinates)[0]['cc']

In [27]:
locations = reverse_geocoder.search(test)

In [36]:
countries = [locations[i]['cc'] for i in range(len(locations))]

In [38]:
df['country'] = countries

In [39]:
df.head

<bound method NDFrame.head of                lat       long  population  wind_speed       solar country
0        83.166667 -74.208333    0.000009    4.253750  107.335396      CA
1        83.166667 -74.166667    0.000011    4.253750  107.335396      CA
2        83.166667 -74.125000    0.000011    4.253750  107.335396      CA
3        83.166667 -74.083333    0.000011    4.253750  107.335396      CA
4        83.166667 -74.041667    0.000008    4.266758  107.335396      CA
...            ...        ...         ...         ...         ...     ...
7550811 -55.916667 -67.250000    0.185715   11.827971         NaN      AR
7550812 -55.916667 -67.083333    0.040128   11.713092         NaN      AR
7550813 -55.958333 -67.333333    0.525298   11.827971         NaN      AR
7550814 -55.958333 -67.291667    1.470422   11.827971         NaN      AR
7550815 -55.958333 -67.250000    0.376293   11.827971         NaN      AR

[7550816 rows x 6 columns]>

In [41]:
df.to_csv('grid.csv',index = False)

In [10]:
grid = pd.read_csv('models/data/grid.csv')
grid.head()

,lat,long,population,wind_speed,solar,country
0,83.166667,-74.208333,0.000009,4.253750,107.335396,CA
1,83.166667,-74.166667,0.000011,4.253750,107.335396,CA
2,83.166667,-74.125000,0.000011,4.253750,107.335396,CA
3,83.166667,-74.083333,0.000011,4.253750,107.335396,CA
4,83.166667,-74.041667,0.000008,4.266758,107.335396,CA


In [9]:
iso = pd.read_csv('all.csv')
iso.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [19]:
df = pd.merge(grid, iso[['name', 'alpha-2']], left_on='country', right_on='alpha-2').drop(['alpha-2'], axis=1)
df.head()

,lat,long,population,wind_speed,solar,country,name
0,83.166667,-74.208333,0.000009,4.253750,107.335396,CA,Canada
1,83.166667,-74.166667,0.000011,4.253750,107.335396,CA,Canada
2,83.166667,-74.125000,0.000011,4.253750,107.335396,CA,Canada
3,83.166667,-74.083333,0.000011,4.253750,107.335396,CA,Canada
4,83.166667,-74.041667,0.000008,4.266758,107.335396,CA,Canada


## Removing countries with conflicts with -9 and -10 intensity

In [24]:
'Canada' in conflict_countries

True

In [25]:
df = df[~df['name'].isin(conflict_countries)]
df.head()

,lat,long,population,wind_speed,solar,country,name
356757,82.875,-64.458333,0.000011,4.604802,124.155983,GL,Greenland
356758,82.875,-64.416667,0.000006,4.604802,124.155983,GL,Greenland
356759,82.875,-64.000000,0.000001,4.647173,124.155983,GL,Greenland
356760,82.875,-63.958333,0.000005,4.647173,124.155983,GL,Greenland
356761,82.875,-63.916667,0.000005,4.647173,124.155983,GL,Greenland


In [ ]:
df.to_csv('grid_conflicts.csv',index = False)